In [ ]:
#Steven Pham
# Homework 4

In [247]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load_json = "purchase_data.json"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_json(file_to_load_json, orient="records")

In [248]:
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [250]:
##### Player Count ######
#########################
# Finding Total Number of Players


# Capturing unique only and using Len to find amount
unique_players_df = purchase_data["SN"].unique()

total_players = len(unique_players_df)

# Output total players using Length function on total_players_df
pd.DataFrame({"Total Players": [total_players]})

,Total Players
0,573


In [251]:
## Purchasing Analysis (Total) ##
#################################
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

# Number of Unique Items
unique_items_df = purchase_data["Item Name"].unique()
total_unique_items = len(unique_items_df)

# Total Number of Purchases
total_purchase = len(purchase_data)

# Total Revenue
total_purchase_price = purchase_data["Price"].sum()

# Average Purchase Price
avg_purchase_price = total_purchase_price/total_purchase

# Setting up table
summary_table = pd.DataFrame({"Number of Unique Items": [total_unique_items],
              "Average Purchase Price": [avg_purchase_price],
              "Total Number of Purchases": [total_purchase],
              "Total Revenue": [total_purchase_price] })


### Formatting Summary table

# Round everything to 2 decimal
summary_table = summary_table.round(2) 

# Setting currency mask
summary_table ["Average Purchase Price"] = summary_table["Average Purchase Price"].map("$ {:,.2f}".format)
summary_table ["Total Revenue"] = summary_table["Total Revenue"].map("$ {:,.2f}".format)


# summary_table ["Total Number of Purchases"] = summary_table["Total Number of Purchases"].map("{:,}".format)


# Reshuffle table to prefered order
summary_table = summary_table.loc[:,["Number of Unique Items", "Average Purchase Price", 
                                     "Total Number of Purchases", "Total Revenue"]]
# Output the summary_table
summary_table

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$ 2.93,780,"$ 2,286.33"


In [252]:
##### Gender Demographics ########
##################################
#  Percentage and Count of Male Players
#  Percentage and Count of Female Players
#  Percentage and Count of Other / Non-Disclosed

# List all unique values in Gender column
purchase_data.Gender.unique()

# Data Frame for only Gender and SN. 
total_demographics = purchase_data.loc[:, ["Gender", "SN"]]

# Drop all dupe SN in table to avoid data skew for gender
total_demographics = total_demographics.drop_duplicates("SN")


# Aggregrate data for gender count and percentage
gender_demographics_total = total_demographics["Gender"].value_counts()
gender_demographics_percent = gender_demographics_total / total_players *100 

# Create table with the following columns
gender_demographics_output = pd.DataFrame({"Total Count": gender_demographics_total, 
                                           "Percentage of Players": gender_demographics_percent})

# Rounding output to 2 decimal
gender_demographics_output = gender_demographics_output.round(2) 

# Output table
gender_demographics_output



,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [253]:
### Purchasing Analysis (Gender) ###
#################################### 
# The below each broken by gender
#    Purchase Count
#    Average Purchase Price
#    Total Purchase Value
#    Normalized Totals

# Data Frame for only Gender, SN, Price. Keep all, including dupes
purchase_analysis_df = purchase_data.loc[:, ["Gender", "SN", "Price", "Age"]]

# Groupby gender for sum, avg, and counts
gender_purchase_total = purchase_analysis_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = purchase_analysis_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_counts = purchase_analysis_df.groupby(["Gender"]).count()["Price"].rename("Purchase Count")

# Divide the purchasing total from prior total output
normalized_total = gender_purchase_total / gender_demographics_output["Total Count"]


# Convert to DataFrame
gender_data = pd.DataFrame({"Purchase Count": gender_counts, 
                            "Average Purchase Price": gender_average, 
                            "Total Purchase Value": gender_purchase_total, 
                            "Normalized Totals": normalized_total})

# Rounding and adding currency mask
gender_data = gender_data.round(2) 
gender_data["Average Purchase Price"] = gender_data["Average Purchase Price"].map("$  {:,.2f}".format)
gender_data["Total Purchase Value"] = gender_data["Total Purchase Value"].map("$  {:,.2f}".format)
gender_data["Normalized Totals"] = gender_data["Normalized Totals"].map("$  {:,.2f}".format)

# Setting up table
gender_data = gender_data.loc[:, ["Purchase Count", "Average Purchase Price", 
                                  "Total Purchase Value", "Normalized Totals"]]

# Output table
gender_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$ 2.82,$ 382.91,$ 3.83
Male,633,$ 2.95,"$ 1,867.68",$ 4.02
Other / Non-Disclosed,11,$ 3.25,$ 35.74,$ 4.47


In [254]:
### Age Demographics ###
########################
# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
#       Purchase Count
#       Average Purchase Price
#       Total Purchase Value
#       Normalized Totals


# Bins + group made
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]


# putting original DF into age bins (research more on this)
purchase_analysis_df["Purchase Count"] = pd.cut(purchase_analysis_df["Age"], age_bins, labels=group_name)
purchase_analysis_df["Average Purchase Price"] = pd.cut(purchase_analysis_df["Price"], age_bins, labels=group_name)


# Purchase counts
age_demographics_totals = purchase_analysis_df["Purchase Count"].value_counts()
avg_purchase = purchase_analysis_df["Average Purchase Price"].value_counts()


# Create Table
age_demographics = pd.DataFrame({"Purchase Count": age_demographics_totals,
                                 "Average Purchase Price": avg_purchase})

# Output table
age_demographics = age_demographics.round(2)
age_demographics.sort_index()



,Average Purchase Price,Purchase Count
10-14,0,35
15-19,0,133
20-24,0,336
25-29,0,125
30-34,0,64
35-39,0,42
40+,0,17
<10,780,28


In [255]:
#### Top Spenders ######
########################
# Identify the the top 5 spenders in the game by 
# total purchase value, then list (in a table):
#     SN
#     Purchase Count
#     Average Purchase Price
#     Total Purchase Value